In [2]:
import pandas as pd
import numpy as np
import os
import glob
import pickle

pd.set_option('max_colwidth', 800)

from datetime import datetime
from datetime import timedelta
from datetime import date

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import contractions
import nltk

import spacy

nltk.download('stopwords')

import re                                  
import string                             
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer 

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import math
#import xlsxwriter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
wnl = nltk.stem.WordNetLemmatizer()

# loading up the language model: English
nlp = spacy.load('en')

# from sklearn.feature_extraction import text
# stop = text.ENGLISH_STOP_WORDS
stop = ['etc', 's', 't', 'ie', 'amp']

lemmatizer = nltk.stem.WordNetLemmatizer()

import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

[nltk_data] Downloading package stopwords to /Users/cgli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "Andre_sales_records_1-50/*.csv"))

In [4]:
file_list = []

for file in csv_files:
    if file.endswith('.csv'):
        df = pd.read_csv(file,sep=",")
        df['filename'] = file
        file_list.append(df)

sales_df = pd.concat(file_list, ignore_index=True)
#all_days.to_csv("all.txt")

In [5]:
#sales_df.head(10)

In [6]:
sales_df['NFTcollection'] = sales_df.filename.str.split('/').str[6]
sales_df['NFTcollection'] = sales_df.NFTcollection.str.split('-').str[0]
sales_df['NFTcollection'] = sales_df['NFTcollection'].str.strip()

In [7]:
sales_df = sales_df[['DateTime', 'Sales (USD)', 'NFTcollection']]

In [8]:
sales_df[sales_df['NFTcollection'] == 'Anonymice'].head(3)

DateTime  Sales (USD) NFTcollection
0  2021-09-16 00:00:00   1125042.78     Anonymice
1  2021-09-17 00:00:00   3934885.69     Anonymice
2  2021-09-18 00:00:00   1747688.82     Anonymice

In [9]:
tweet_df = pd.read_csv('frame_1-50_100-150col.csv', index_col=None, header=0)

<ipython-input-9-9ce646bca2d1>:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  tweet_df = pd.read_csv('frame_1-50_100-150col.csv', index_col=None, header=0)


In [10]:
#tweet_df.head(3)

In [11]:
Anonymice_df = tweet_df[tweet_df['Anonymice'] == 1][['text', 'created_at', 'retweets']].reset_index(drop=True)

In [12]:
Anonymice_df.head(2)

text  \
0  @greatmando_nft @mypethooligan @GoGalaGames @robotosNFT @DeadfellazNFT @RealSupDucks @FLUF_World @worldofwomennft @CoolmanUniverse @Worldwide_WEB3 @capsule_house @BYOPills @adidasoriginals @nuclearnerds @alienfrens Anonymice   
1                                                                                              @PullinOut4Life @instanceseth @doodles @AnonymiceNFT @alienfrens @nftworldsNFT @SmolBrainsNFT @EtherOrcs Yup, I’m deep in the horde.   

                  created_at  retweets  
0  2022-01-03 05:04:57+00:00       0.0  
1  2022-01-03 01:21:45+00:00       0.0

In [13]:
Anonymice_df.shape

(2503, 3)

In [14]:
# Anonymice_df['Tweet_nospecial'] = Anonymice_df["text"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
# #Anonymice_df['Tweet_noURL'] = Anonymice_df['Tweet_nospecial'].apply(lambda x: re.split('https?:\/\/.*', str(x))[0])
# Anonymice_df['Tweet_noURL'] = Anonymice_df['Tweet_nospecial'].str.replace('http\S+','')
# Anonymice_df['Tweet_noMentions'] = Anonymice_df['Tweet_noURL'].str.replace(r'@[A-Za-z0-9]+', '', regex=True)
# Anonymice_df["Tweet_expanded"]=Anonymice_df['Tweet_noMentions'].apply(lambda x: [contractions.fix(word) for word in x.split()])
# Anonymice_df['Tweet_expanded'] = [' '.join(map(str, l)) for l in Anonymice_df['Tweet_expanded']]
# Anonymice_df["Tweet_noNewLine"]= Anonymice_df['Tweet_expanded'].replace(r'\\n', ' ', regex=True)
# Anonymice_df["Tweet_noPunctuation"] = Anonymice_df['Tweet_noNewLine'].str.replace('[^\w\s]','')
# Anonymice_df["Tweet_noPunctuation"] = Anonymice_df['Tweet_noPunctuation'].str.replace('[_]','')
# Anonymice_df["Tweet_noNumbers"] = Anonymice_df['Tweet_noPunctuation'].str.replace(r'[0-9]',' ')
# Anonymice_df['Tweet_lowercase'] = Anonymice_df.Tweet_noNumbers.str.lower()
# Anonymice_df['Tweet_noStopwords'] = Anonymice_df['Tweet_lowercase'].apply(lambda x: [item for item in x.split() if item not in stop])
# Anonymice_df['Tweet_lemmatized'] = Anonymice_df['Tweet_noStopwords'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
# Anonymice_df = Anonymice_df[['text','created_at', 'Tweet_lemmatized']]
# Anonymice_df['Tweet_lemmatized'] = [' '.join(map(str, l)) for l in Anonymice_df['Tweet_lemmatized']]
# Anonymice_df.tail(3)

In [15]:
def clean_tweets(df):
    #remove special characters
    df['Tweet_nospecial'] = df["text"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    #Remove hyperlinks
    df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
    #Remove mentions
    df['Tweet_noMentions'] = df['Tweet_noURL'].str.replace(r'@[A-Za-z0-9_]+', '', regex=True)
    #Expand contractions
    df["Tweet_expanded"]=df['Tweet_noMentions'].apply(lambda x: [contractions.fix(word) for word in x.split()])
    df['Tweet_expanded'] = [' '.join(map(str, l)) for l in df['Tweet_expanded']]
    #remove newline
    df["Tweet_noNewLine"]= df['Tweet_expanded'].replace(r'\\n', ' ', regex=True)
    #Remove punctuation
    df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
    df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replace('[_]','')
    #Remove numbers
    df["Tweet_noNumbers"] = df['Tweet_noPunctuation'].str.replace(r'[0-9]',' ')
    #Make lowercase
    df['Tweet_lowercase'] = df.Tweet_noNumbers.str.lower()
    #remove stopwords
    df['Tweet_noStopwords'] = df['Tweet_lowercase'].apply(lambda x: [item for item in x.split() if item not in stop])
    #lemmatize words
    df['Tweet_lemmatized'] = df['Tweet_noStopwords'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
    #subset columns
    df = df[['text','created_at', 'retweets', 'Tweet_lemmatized']]
    df['Tweet_lemmatized'] = [' '.join(map(str, l)) for l in df['Tweet_lemmatized']]
    
    return df

In [16]:
Anonymice_df = clean_tweets(Anonymice_df)

<ipython-input-15-ee14c66a9716>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-15-ee14c66a9716>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-15-ee14c66a9716>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replace('[_]','')
<ipython-input-15-ee14c66a9716>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noNumbers"] = df['Tweet_noPunctuation'].str.replace(r'[0-9]',' ')
<ipython-input-15-ee14c66a9716>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [17]:
#Anonymice_df.head(30)

In [18]:
#Anonymice_df[Anonymice_df['Tweet_lemmatized'] == '']

In [19]:
#pip install flair

In [20]:
# import flair
# sentiment_model = flair.models.TextClassifier.load('en-sentiment')

In [21]:
# probs = []
# sentiments = []

# # use regex expressions (in clean function) to clean tweets
# #Anonymice_df['text'] = Anonymice_df['text'].apply(clean)

# for tweet in Anonymice_df['Tweet_lemmatized'].to_list(): 
#     # make prediction
#     sentence = flair.data.Sentence(tweet)
#     sentiment_model.predict(sentence)
#     # extract sentiment prediction
#     probs.append(sentence.labels[0].score)  # numerical score 0-1
#     sentiments.append(sentence.labels[0].value)  # 'POSITIVE' or 'NEGATIVE'

# # add probability and sentiment predictions to tweets dataframe
# Anonymice_df['probability'] = probs
# Anonymice_df['sentiment'] = sentiments

In [22]:
#Anonymice_df.head(5)

In [23]:
#graph first week of sales data for anonymice

#Anonymice_df[Anonymice_df['sentiment'] == 'NEGATIVE']

In [24]:
Anonymice_df['created_at'] = Anonymice_df.created_at.str.split(' ').str[0]

In [25]:
Anonymice_df.head(2)

text  \
0  @greatmando_nft @mypethooligan @GoGalaGames @robotosNFT @DeadfellazNFT @RealSupDucks @FLUF_World @worldofwomennft @CoolmanUniverse @Worldwide_WEB3 @capsule_house @BYOPills @adidasoriginals @nuclearnerds @alienfrens Anonymice   
1                                                                                              @PullinOut4Life @instanceseth @doodles @AnonymiceNFT @alienfrens @nftworldsNFT @SmolBrainsNFT @EtherOrcs Yup, I’m deep in the horde.   

   created_at  retweets           Tweet_lemmatized  
0  2022-01-03       0.0                  anonymice  
1  2022-01-03       0.0  yup i m deep in the horde

In [29]:
type(Anonymice_df['created_at'][0])

str

In [26]:
# def sentiment2numeric(row):
#     if row['sentiment'] == 'POSITIVE':
#         return 1
#     else:
#         return 0

In [27]:
#Anonymice_df['sentiment_num'] = Anonymice_df.apply(sentiment2numeric, axis=1)

In [ ]:
#Anonymice_df.head(3)

In [21]:
def sentimentScore(Tweet):
    analyzer = SentimentIntensityAnalyzer()
    score_values = []
    for tweet in Anonymice_df['Tweet_lemmatized']:
        vs1 = analyzer.polarity_scores(tweet)
        print("Score: " + str(vs1))
        score_values.append(vs1)
    return score_values

In [22]:
sentiment_scores = pd.DataFrame(sentimentScore(Anonymice_df['Tweet_lemmatized']))

Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 0.709, 'pos': 0.291, 'compound': 0.6249}
Score: {'neg': 0.123, 'neu': 0.877, 'pos': 0.0, 'compound': -0.4215}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.146, 'neu': 0.735, 'pos': 0.118, 'compound': -0.2217}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 0.805, 'pos': 0.195, 'compound': 0.4404}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 0.855, 'pos': 0.145, 'comp

Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 0.81, 'pos': 0.19, 'compound': 0.8271}
Score: {'neg': 0.0, 'neu': 0.899, 'pos': 0.101, 'compound': 0.5423}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.08, 'neu': 0.92, 'pos': 0.0, 'compound': -0.3612}
Score: {'neg': 0.073, 'neu': 0.788, 'pos': 0.139, 'compound': 0.34}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'compound': 0.5859}
Score: {'neg': 0.0, 'neu': 0.444, 'pos': 0.556, 'compound': 0.3612}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'c

Score: {'neg': 0.0, 'neu': 0.539, 'pos': 0.461, 'compound': 0.7717}
Score: {'neg': 0.0, 'neu': 0.493, 'pos': 0.507, 'compound': 0.7351}
Score: {'neg': 0.393, 'neu': 0.427, 'pos': 0.179, 'compound': -0.5423}
Score: {'neg': 0.0, 'neu': 0.858, 'pos': 0.142, 'compound': 0.5106}
Score: {'neg': 0.07, 'neu': 0.655, 'pos': 0.275, 'compound': 0.5719}
Score: {'neg': 0.0, 'neu': 0.69, 'pos': 0.31, 'compound': 0.6597}
Score: {'neg': 0.0, 'neu': 0.733, 'pos': 0.267, 'compound': 0.743}
Score: {'neg': 0.146, 'neu': 0.854, 'pos': 0.0, 'compound': -0.4404}
Score: {'neg': 0.0, 'neu': 0.847, 'pos': 0.153, 'compound': 0.4404}
Score: {'neg': 0.195, 'neu': 0.805, 'pos': 0.0, 'compound': -0.4404}
Score: {'neg': 0.0, 'neu': 0.711, 'pos': 0.289, 'compound': 0.9081}
Score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Score: {'neg': 0.0, 'neu': 0.377, 'pos': 0.623, 'compound': 0.5106}
Score: {'neg': 0.0, 'neu': 0.762, 'pos': 0.238, 'compound': 0.7612}
Score: {'neg': 0.0, 'neu': 0.633, 'pos': 0.367, 'com

In [23]:
Anonymice_df = pd.merge(Anonymice_df, sentiment_scores, left_index=True, right_index=True)

In [24]:
Anonymice_df.head(3)

text  \
0  @greatmando_nft @mypethooligan @GoGalaGames @robotosNFT @DeadfellazNFT @RealSupDucks @FLUF_World @worldofwomennft @CoolmanUniverse @Worldwide_WEB3 @capsule_house @BYOPills @adidasoriginals @nuclearnerds @alienfrens Anonymice   
1                                                                                              @PullinOut4Life @instanceseth @doodles @AnonymiceNFT @alienfrens @nftworldsNFT @SmolBrainsNFT @EtherOrcs Yup, I’m deep in the horde.   
2                                                                                                      @instanceseth @jxmjxmblank @doodles @AnonymiceNFT @alienfrens @nftworldsNFT @SmolBrainsNFT @EtherOrcs He owns orcs I believe   

   created_at  retweets           Tweet_lemmatized  neg  neu  pos  compound  
0  2022-01-03       0.0                  anonymice  0.0  1.0  0.0       0.0  
1  2022-01-03       0.0  yup i m deep in the horde  0.0  1.0  0.0       0.0  
2  2022-01-03       0.0     he owns orcs i believe  0.0  1.0  0.0       0.0

In [ ]:
#change this to if the retweets is 0, multiply by 1, but if > 0 then multiply by # retweets

#Anonymice_df['sentiment_vader_score'] = Anonymice_df['compound']*Anonymice_df['retweets']


In [25]:
def calc_vader_sentiment(row):
    if row['retweets'] == 0:
        return row['compound']
    else:
        return row['compound']*row['retweets']

In [26]:
Anonymice_df['sentiment_vader_score'] = Anonymice_df.apply(calc_vader_sentiment, axis=1)

In [27]:
Anonymice_df.head(30)

text  \
0                                                                        @greatmando_nft @mypethooligan @GoGalaGames @robotosNFT @DeadfellazNFT @RealSupDucks @FLUF_World @worldofwomennft @CoolmanUniverse @Worldwide_WEB3 @capsule_house @BYOPills @adidasoriginals @nuclearnerds @alienfrens Anonymice   
1                                                                                                                                                                    @PullinOut4Life @instanceseth @doodles @AnonymiceNFT @alienfrens @nftworldsNFT @SmolBrainsNFT @EtherOrcs Yup, I’m deep in the horde.   
2                                                                                                                                                                            @instanceseth @jxmjxmblank @doodles @AnonymiceNFT @alienfrens @nftworldsNFT @SmolBrainsNFT @EtherOrcs He owns orcs I believe   
3                                                                                                                                                                 @jxmjxmblank @doodles @AnonymiceNFT @alienfrens @nftworldsNFT @SmolBrainsNFT we have a list of great white buffalos ser, except doodles   
4                                                                                 My 2022 most bullish projects list, but from projects that I am not currently invested in. No self pump prediction list here.\n\n1. @doodles \n2. @AnonymiceNFT \n3. @alienfrens \n4. @nftworldsNFT \n5. @SmolBrainsNFT   
5                @greatmando_nft @mypethooligan @GoGalaGames @robotosNFT @DeadfellazNFT @RealSupDucks @FLUF_World @worldofwomennft @CoolmanUniverse @Worldwide_WEB3 @capsule_house @BYOPills @adidasoriginals @nuclearnerds @alienfrens @AnonymiceNFT &amp; @chain_runners are a must as Onchain holdings   
6                                                                                                                                                                  📈 #0x92C bought Anonymice #1043 - Anonymice  by AnonymiceNFT for $35.2K (~9.99 ETH)\n\nhttps://t.co/F0nIpSnelJ https://t.co/RPahOAanP2   
7                                                                                                                                                                   📈 #0x9eE bought Anonymice #7828 - Anonymice  by AnonymiceNFT for $34.9K (~9.9 ETH)\n\nhttps://t.co/yrY3ROtOBA https://t.co/RLdqjsXhJH   
8                                                                                                                                                                   📈 #0x9eE bought Anonymice #4944 - Anonymice  by AnonymiceNFT for $34.8K (~9.9 ETH)\n\nhttps://t.co/tbramsguGA https://t.co/UuYYwlfvoQ   
9                                                                                                                                              @andymilonakis "What makes you think I won't cutchu!?" Love that movie. Awesome pulls on the Anonymice, want one SO bad but they jumped. out of my league!   
10                                                                                                                  NFT Name: Anonymice #7074 \nEther price: 9.86 Ether \nEstimated USD Price: $34898.98\nEtherscan: https://t.co/cifV7unpCM\n#NFT #crypto #opensea #NFTCommunity https://t.co/I2ZgkxqsJz   
11                                                                                                                                                                 📈 #0x85b bought Anonymice #7074 - Anonymice  by AnonymiceNFT for $34.7K (~9.86 ETH)\n\nhttps://t.co/OEv3LtZTSK https://t.co/p5fCsTN8Tw   
12                                                                                                                                                                                                                @economist @jasvault Anonymice is, and surely the kongz. Waiting for Milk now let's see   
13                                                                             

In [28]:
Anonymice_df_daily_mean=(Anonymice_df.groupby(Anonymice_df.created_at).mean()).reset_index()
Anonymice_df_daily_mean.head(3)

created_at  retweets    neg    neu    pos  compound  sentiment_vader_score
0  2021-08-02       6.0  0.188  0.812  0.000   -0.8402                -5.0412
1  2021-08-20       0.0  0.000  0.872  0.128    0.3612                 0.3612
2  2021-08-31       0.0  0.000  1.000  0.000    0.0000                 0.0000

In [29]:
Anonymice_df_daily_count = Anonymice_df.groupby('created_at')['text'].count().to_frame(name = 'num_tweets').reset_index()
Anonymice_df_daily_count.head(5)

created_at  num_tweets
0  2021-08-02           1
1  2021-08-20           1
2  2021-08-31           1
3  2021-09-13           1
4  2021-09-14           2

In [30]:
Anonymice_mean_count = pd.merge(Anonymice_df_daily_mean, Anonymice_df_daily_count, on='created_at')
Anonymice_mean_count.head(6)

created_at  retweets       neg       neu       pos  compound  \
0  2021-08-02  6.000000  0.188000  0.812000  0.000000 -0.840200   
1  2021-08-20  0.000000  0.000000  0.872000  0.128000  0.361200   
2  2021-08-31  0.000000  0.000000  1.000000  0.000000  0.000000   
3  2021-09-13  1.000000  0.000000  0.891000  0.109000  0.510600   
4  2021-09-14  3.000000  0.026000  0.799000  0.175500  0.729200   
5  2021-09-16  0.565217  0.005478  0.822913  0.171609  0.333565   

   sentiment_vader_score  num_tweets  
0                -5.0412           1  
1                 0.3612           1  
2                 0.0000           1  
3                 0.5106           1  
4                 2.1876           2  
5                 0.5720          23

In [31]:
#multiple avg vader score per day by # of tweets

Anonymice_mean_count['final_vader_score'] = Anonymice_mean_count['sentiment_vader_score']*Anonymice_mean_count['num_tweets']
Anonymice_mean_count.head(8)

created_at  retweets       neg       neu       pos  compound  \
0  2021-08-02  6.000000  0.188000  0.812000  0.000000 -0.840200   
1  2021-08-20  0.000000  0.000000  0.872000  0.128000  0.361200   
2  2021-08-31  0.000000  0.000000  1.000000  0.000000  0.000000   
3  2021-09-13  1.000000  0.000000  0.891000  0.109000  0.510600   
4  2021-09-14  3.000000  0.026000  0.799000  0.175500  0.729200   
5  2021-09-16  0.565217  0.005478  0.822913  0.171609  0.333565   
6  2021-09-17  0.772152  0.036658  0.830013  0.133316  0.195795   
7  2021-09-18  3.237500  0.064238  0.852237  0.083475  0.022183   

   sentiment_vader_score  num_tweets  final_vader_score  
0              -5.041200           1            -5.0412  
1               0.361200           1             0.3612  
2               0.000000           1             0.0000  
3               0.510600           1             0.5106  
4               2.187600           2             4.3752  
5               0.572000          23            13.1560  
6               0.665918          79            52.6075  
7               1.722869          80           137.8295

In [32]:
Anonymice_sentiment_df = Anonymice_mean_count[['created_at', 'final_vader_score']]
Anonymice_sentiment_df.head(6)

created_at  final_vader_score
0  2021-08-02            -5.0412
1  2021-08-20             0.3612
2  2021-08-31             0.0000
3  2021-09-13             0.5106
4  2021-09-14             4.3752
5  2021-09-16            13.1560

**Preparing sales data**

In [33]:
Anonymice_sales_df = sales_df[sales_df['NFTcollection'] == 'Anonymice']
Anonymice_sales_df['DateTime'] = Anonymice_sales_df.DateTime.str.split(' ').str[0]
Anonymice_sales_df = Anonymice_sales_df.rename(columns={"DateTime": "created_at"})
Anonymice_sales_df = Anonymice_sales_df.rename(columns={"Sales (USD)": "Sales"})
Anonymice_sales_df.head(3)

<ipython-input-33-d3d6a1b9c650>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Anonymice_sales_df['DateTime'] = Anonymice_sales_df.DateTime.str.split(' ').str[0]


created_at       Sales NFTcollection
0  2021-09-16  1125042.78     Anonymice
1  2021-09-17  3934885.69     Anonymice
2  2021-09-18  1747688.82     Anonymice

In [34]:
scaler = StandardScaler()
Anonymice_sales_df['prices_scaled'] = scaler.fit_transform(Anonymice_sales_df[['Sales']])
Anonymice_sales_df.head(2)

created_at       Sales NFTcollection  prices_scaled
0  2021-09-16  1125042.78     Anonymice       1.232625
1  2021-09-17  3934885.69     Anonymice       5.555677

In [35]:
Anonymice_sales_df = Anonymice_sales_df[['created_at', 'prices_scaled']]

In [36]:
Anonymice_merged = pd.merge(Anonymice_sentiment_df, Anonymice_sales_df, on='created_at')
Anonymice_merged.head(6)

created_at  final_vader_score  prices_scaled
0  2021-09-16            13.1560       1.232625
1  2021-09-17            52.6075       5.555677
2  2021-09-18           137.8295       2.190590
3  2021-09-19            15.5996       7.842699
4  2021-09-20            49.4080       2.627685
5  2021-09-21             8.0060       1.251632

In [37]:
Anonymice_merged.shape

(109, 3)

In [38]:
Anonymice_merged['previous_day_sentiment'] = Anonymice_merged['final_vader_score'].shift(1)

In [39]:
Anonymice_merged.head(3)

created_at  final_vader_score  prices_scaled  previous_day_sentiment
0  2021-09-16            13.1560       1.232625                     NaN
1  2021-09-17            52.6075       5.555677                 13.1560
2  2021-09-18           137.8295       2.190590                 52.6075

**Predictive Models: LSTM, ARIMA, NeuralProphet**

In [41]:
training_set_index = int(0.8*Anonymice_merged.shape[0])

In [55]:
training_set = Anonymice_merged.iloc[:training_set_index, 0:4]
test_set = Anonymice_merged.iloc[training_set_index:, 0:4]

In [56]:
training_set

created_at  final_vader_score  prices_scaled  previous_day_sentiment
0   2021-09-16            13.1560       1.232625                     NaN
1   2021-09-17            52.6075       5.555677                 13.1560
2   2021-09-18           137.8295       2.190590                 52.6075
3   2021-09-19            15.5996       7.842699                137.8295
4   2021-09-20            49.4080       2.627685                 15.5996
..         ...                ...            ...                     ...
82  2021-12-27             1.1221      -0.370032                  0.0000
83  2021-12-28             1.5318      -0.399391                  1.1221
84  2021-12-31            -0.1208      -0.420559                  1.5318
85  2022-01-02             3.8231       0.071040                 -0.1208
86  2022-01-03             0.7529      -0.408333                  3.8231

[87 rows x 4 columns]

In [57]:
#add a feature - shift 1 day sentiment before
#compare rms/predictability across all 100 collections

In [60]:
sentiment_feature = "previous_day_sentiment"
features = [sentiment_feature, "prices_scaled"]
training_set = training_set[features].dropna()

In [61]:
training_set

previous_day_sentiment  prices_scaled
1                  13.1560       5.555677
2                  52.6075       2.190590
3                 137.8295       7.842699
4                  15.5996       2.627685
5                  49.4080       1.251632
..                     ...            ...
82                  0.0000      -0.370032
83                  1.1221      -0.399391
84                  1.5318      -0.420559
85                 -0.1208       0.071040
86                  3.8231      -0.408333

[86 rows x 2 columns]

In [62]:
X_train = training_set[sentiment_feature]
X_train = X_train.values

In [63]:
X_test = test_set[sentiment_feature]
X_test = X_test.values

In [64]:
y_train = training_set["prices_scaled"]
y_train = y_train.values

In [65]:
y_test = test_set["prices_scaled"]
y_test = y_test.values

In [66]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [67]:
X_test, y_test = np.array(X_test), np.array(y_test)

In [68]:
X_train = np.reshape(X_train, (X_train.shape[0],1))

In [69]:
X_test = np.reshape(X_test, (X_test.shape[0],1))

In [208]:
np.random.seed(1234)
tf.random.set_seed(1234)

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(rate = 0.3))

model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(rate = 0.3))

model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(rate = 0.3))

model.add(LSTM(units=50, return_sequences = False))
model.add(Dropout(rate = 0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [209]:
model.fit(X_train, y_train, epochs=150, batch_size=42
          , verbose=1)

Epoch 1/150
3/3 [==============================] - 5s 5ms/step - loss: 1.7607
Epoch 2/150
3/3 [==============================] - 0s 7ms/step - loss: 1.7498
Epoch 3/150
3/3 [==============================] - 0s 6ms/step - loss: 1.7404
Epoch 4/150
3/3 [==============================] - 0s 6ms/step - loss: 1.7309
Epoch 5/150
3/3 [==============================] - 0s 6ms/step - loss: 1.7190
Epoch 6/150
3/3 [==============================] - 0s 7ms/step - loss: 1.7129
Epoch 7/150
3/3 [==============================] - 0s 7ms/step - loss: 1.7022
Epoch 8/150
3/3 [==============================] - 0s 6ms/step - loss: 1.6883
Epoch 9/150
3/3 [==============================] - 0s 6ms/step - loss: 1.6727
Epoch 10/150
3/3 [==============================] - 0s 6ms/step - loss: 1.6519
Epoch 11/150
3/3 [==============================] - 0s 6ms/step - loss: 1.6133
Epoch 12/150
3/3 [==============================] - 0s 6ms/step - loss: 1.5884
Epoch 13/150
3/3 [==============================] - 0s 6ms/st

In [210]:
y_pred = model.predict(X_test) 
#y_pred = scaler.inverse_transform(y_pred) 

In [211]:
y_pred.shape

(22, 1)

In [212]:
rms = mean_squared_error(y_test, y_pred, squared=False)

In [213]:
rms

0.6089429133303677

In [159]:
y_pred = scaler.inverse_transform(y_pred) 

In [1]:
y_pred

NameError: name 'y_pred' is not defined

In [160]:
np.mean(y_pred)

337598.06

In [161]:
y_test_prices = scaler.inverse_transform(y_test) 

In [162]:
np.mean(y_test_prices)

107390.66409090908

In [163]:
np.mean(y_train)

0.3457762473108301

In [164]:
rms_2 = mean_squared_error(y_test, np.repeat(0.35596990639184894, 22), squared=False)

In [165]:
rms_2

0.7202845651433396

In [ ]:
rms_2 = mean_squared_error(y_test, np.repeat(np.mean(y_train), len(y_test)), squared=False)

ARIMA model

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model=ARIMA(training_set["final_vader_score"],order=(1,1,1))
model_fit=model.fit() 
model_fit.summary()

In [ ]:
Anonymice_group_mean_df = Anonymice_df.groupby('created_at')['sentiment_num'].mean().to_frame(name = 'avg_sentiment').reset_index()

Anonymice_group_mean_df.head(30)

In [ ]:
Anonymice_group_count_df = Anonymice_df.groupby('created_at')['text'].count().to_frame(name = 'num_tweets').reset_index()
Anonymice_group_count_df.head(30)

In [ ]:
# Loading the dictionary with minting dates 
with open('data/mint_dict.pkl', 'rb') as f:
    mint_dict = pickle.load(f)
    
print(len(mint_dict.keys()))

In [ ]:
mint_dict['Anonymice']

In [ ]:
merged1 = pd.merge(Anonymice_group_mean_df, Anonymice_group_count_df, on='created_at')
final_merged = pd.merge(merged1, Anonymice_sales_df, on='created_at')
final_merged['created_at'] = pd.to_datetime(final_merged['created_at']).dt.date
final_merged.head(50)

In [ ]:
final_merged = final_merged[(final_merged['created_at'] >= mint_dict['Anonymice'].date()) & (final_merged['created_at'] <= mint_dict['Anonymice'].date() + timedelta(days=30))]


In [ ]:
# #fig,ax = plt.subplots()

# import matplotlib.pyplot as plt

# # Plot a simple line chart
# plt.plot(final_merged['created_at'], final_merged['avg_sentiment'], color='g', label='Average sentiment')

# # Plot another line on the same chart/graph
# plt.plot(final_merged['created_at'], final_merged['Sales (USD)'], color='r', label='Sales')

# plt.legend()
# plt.show()

# ax.set_xlabel("Date")
# #ax.set_ylabel("weight")
# ax.legend(loc='best')

In [ ]:
import matplotlib.pyplot as plt 

fig, ax = plt.subplots(figsize=(20,10)) # Create the figure and axes object
#f, axs = plt.subplots(2,2,figsize=(15,15))
#ax.set_xticklabels(labels = final_merged['created_at'], rotation=45)

# Plot the first x and y axes:
final_merged.plot(x = 'created_at', y = 'avg_sentiment', ax = ax) 
# Plot the second x and y axes. By secondary_y = True a second y-axis is requested:
# (see https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html for details)
final_merged.plot(x = 'created_at', y = 'Sales (USD)', ax = ax, secondary_y = True) 
